# How to open result (MS Windows OS only)

This tutorial demonstrates how to open and review results using workflow method.

## Prerequisites

### Perform imports

In [1]:
import os
from pathlib import Path

from ansys.speos.core import Project, Speos, launcher
from ansys.speos.core.kernel.client import (
    default_docker_channel,
)
from ansys.speos.core.simulation import SimulationDirect


### Define constants
Constants help ensure consistency and avoid repetition throughout the example.

In [2]:
HOSTNAME = "localhost"
GRPC_PORT = 50098  # Be sure the Speos GRPC Server has been started on this port.
FILE_NAME = "LG_50M_Colorimetric_short.sv5"
RESULT_NAME = "ASSEMBLY1.DS (0).Dom Irradiance Sensor (0).xmp"
USE_DOCKER = True  # Set to False if you're running this example locally as a Notebook.
USE_GPU = False

## Model Setup

### Load assets
The assets used to run this example are available in the
[PySpeos repository](https://github.com/ansys/pyspeos/) on GitHub.

> **Note:** Make sure you
> have downloaded simulation assets and set ``assets_data_path``
> to point to the assets folder.

In [3]:
if USE_DOCKER:  # Running on the remote server.
    assets_data_path = Path("/app") / "assets"
else:
    assets_data_path = Path("/path/to/your/download/assets/directory")

### Connect to the RPC Server
This Python client connects to a server where the Speos engine
is running as a service. In this example, the server and
client are the same
machine.

In [4]:
if USE_DOCKER:
    speos = Speos(channel=default_docker_channel())
else:
    speos = launcher.launch_local_speos_rpc_server(port=GRPC_PORT)

/home/runner/work/pyspeos/pyspeos/.venv/lib/python3.10/site-packages/ansys/tools/common/cyberchannel.py:188: UserWarning: Starting gRPC client without TLS on localhost:50098. This is INSECURE. Consider using a secure connection.
  warn(f"Starting gRPC client without TLS on {target}. This is INSECURE. Consider using a secure connection.")


### Create project from a Speos file

The ``Project`` class is instantiated by passing a ``Speos`` instance and the name of the Speos
project file.

In [5]:
p = Project(
    speos=speos,
    path=str(assets_data_path / FILE_NAME / FILE_NAME),
)
print(p)

/home/runner/work/pyspeos/pyspeos/.venv/lib/python3.10/site-packages/ansys/speos/core/project.py:760: UserWarning: The pySpeos feature : FaceStub.read_batch needs a Speos Version of 2025 R2 SP0 or higher.
  f_data_list = face_db.read_batch(refs=f_links)
/home/runner/work/pyspeos/pyspeos/.venv/lib/python3.10/site-packages/ansys/speos/core/project.py:866: UserWarning: The pySpeos feature : SourceSurface needs a Speos Version of 2025 R2 SP0 or higher.
  src_feat = SourceSurface(
/home/runner/work/pyspeos/pyspeos/.venv/lib/python3.10/site-packages/ansys/speos/core/project.py:938: UserWarning: The pySpeos feature : SimulationDirect needs a Speos Version of 2025 R2 SP0 or higher.
  sim_feat = SimulationDirect(


{
    "name": "LG_50M_Colorimetric_short",
    "description": "From /app/assets/LG_50M_Colorimetric_short.sv5/LG_50M_Colorimetric_short.sv5",
    "part_guid": "8d93f87b-8181-4aa3-adf5-09a6ba012587",
    "sources": [
        {
            "name": "Dom Source 2 (0) in SOURCE2",
            "metadata": {
                "UniqueId": "754c4219-e055-4150-a311-f0dd015b181a"
            },
            "source_guid": "17a6c643-eacc-4c6f-bfaa-d1e8282c087b",
            "description": "",
            "source": {
                "name": "Dom Source 2 (0) in SOURCE2",
                "surface": {
                    "radiant_flux": {
                        "radiant_value": 6.590041607465698
                    },
                    "intensity_guid": "71b7f80b-7283-452e-9859-8f99cae770b9",
                    "exitance_constant": {
                        "geo_paths": [
                            {
                                "geo_path": "Solid Body in SOURCE2:2920204960/Face in SOURCE2:222",

### Retrieve the simulation feature

Use the method ``Project.find()`` to retrieve an instance
of the ``SimulationDirect`` feature.

In [6]:
sim = p.find(name=".*", name_regex=True, feature_type=SimulationDirect)[0]

## Run simulation

The simulation can be run using either the CPU or with GPU acceleration. The following cell shows
how Python is used to assign the appropriate method to ``run_sim``.

In [7]:
run_sim = sim.compute_GPU if USE_GPU else sim.compute_CPU
results = run_sim()  # run the simulation
print(results)

[upload_response {
  info {
    uri: "c9501a68-dae8-440c-8f38-d375705e71ef"
    file_name: "ASSEMBLY1.DS (0).Dom Irradiance Sensor (0).xmp"
    file_size: 2159432
  }
  upload_duration {
    nanos: 2885964
  }
}
, upload_response {
  info {
    uri: "348b3cd6-2fe7-4ce2-931a-1c86eb34057d"
    file_name: "ASSEMBLY1.DS (0).html"
    file_size: 648130
  }
  upload_duration {
    nanos: 929657
  }
}
]


## Postprocessing
### Open the results (MS Windows OS only):

Display one result as image.

A full path can be given, or the name of the result.

In [8]:
if os.name == "nt":  # Are we running on Windows OS?
    from ansys.speos.core.workflow.open_result import open_result_image

    open_result_image(simulation_feature=sim, result_name=RESULT_NAME)

### Display the image

Display one result in a result viewer.

A full path can be given, or the name of the result.

In [9]:
if os.name == "nt":
    from ansys.speos.core.workflow.open_result import open_result_in_viewer

    open_result_in_viewer(
        simulation_feature=sim,
        result_name=RESULT_NAME,
    )
speos.close()

True